# Map of London

As its looking like most jobs for data science are centered around London, I figured i'd up end up there at some point. Me being the curious person I am, I wanted to get some simple heat map style data on rental prices for the Boroughs, as well as Crime data. 

Although I'm more use to GIS in R, I figured i'd give it a go!

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as mp
